In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../../../")
%load_ext autoreload
%autoreload 2

In [3]:
california_ozone = pd.read_csv("../dataset/cal_o3.csv", index_col = None)

In [4]:
cadf = california_ozone

cadf = cadf[cadf['Time Local'] <= '18:00']
cadf = cadf[cadf['Time Local'] >= '10:00']

In [5]:
county_codes = cadf['County Code'].values
site_code = cadf['Site Num'].values
site_id = [str(county_codes[i]) + str(site_code[i]) for i in range(len(county_codes))]

In [6]:
cadf['ID'] = site_id

In [7]:
cadf.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Date Local',
       'Time Local', 'Date GMT', 'Time GMT', 'Sample Measurement',
       'Units of Measure', 'MDL', 'Uncertainty', 'Qualifier', 'Method Type',
       'Method Code', 'Method Name', 'State Name', 'County Name',
       'Date of Last Change', 'ID'],
      dtype='object')

In [8]:
mask = cadf.groupby(['ID', 'Date Local'])['Sample Measurement'].count() >= 8

In [9]:
cadf = cadf.groupby(['ID', 'Date Local']).mean()[mask]

In [10]:
cadf.reset_index(inplace=True)

In [11]:
cadf = cadf[['ID', 'Date Local', 'Latitude', 'Longitude', 'Sample Measurement']]

In [12]:
m1 = cadf['Date Local'] >= '1990-05-01'
m2 = cadf['Date Local'] <= '1990-09-30'
mf = m1 & m2
cadf = cadf[mf]

In [13]:
m4 = cadf.groupby('ID').count()>=115

In [14]:
cadf = cadf.groupby('ID').mean()[m4]

In [15]:
cadf.dropna(inplace = True)

In [16]:
ca_df = cadf.copy()

## Summary Statistics for Kriging Interpolation in California

In [17]:
ca_df

,Latitude,Longitude,Sample Measurement
ID,,,
1012,38.848232,-121.515236,0.046687
1013,39.138773,-121.618549,0.043239
1072002,36.332179,-119.291228,0.065835
1075,36.493557,-118.829543,0.072096
1076,36.566128,-118.777700,0.067197
...,...,...,...
974001,38.295747,-122.457758,0.037412
991003,37.522712,-120.867707,0.056614
991004,37.371047,-121.133265,0.054918


In [18]:
import geopandas as gpd
g_df = gpd.read_file("../block_groups/california/tl_2016_06_bg.shp")

In [19]:
g_df['INTPTLAT'].max(), g_df['INTPTLAT'].min(), 

('+41.9466483', '+32.5447420')

In [20]:
shapes = g_df['geometry'].unique()
centroids = [shapes[i].centroid.wkt for i in range(len(shapes))]
centroids = [centroids[i].split(' ')[1:] for i in range(len(centroids))]
centroids = [[np.float64(centroids[i][0][1:]), np.float64(centroids[i][1][:-1])] for i in range(len(centroids))]
centroids = np.array(centroids)

In [21]:
centroids.min(axis=0), centroids.max(axis=0)

(array([-124.27033744,   32.54474204]), array([-114.29862718,   41.94353816]))

In [22]:
X = ca_df[['Longitude', 'Latitude']].values
y = ca_df[['Sample Measurement']].values*1000

In [23]:
from polire.interpolate import Kriging
## PyKrige takes Longitude Values in the Range 0 to 360 => Add 360 to our values!
X_krig = X.copy()
X_test_krig = centroids.copy()
# X_krig[:, 0] = X_krig[:, 0] + 360
krig = Kriging(variogram_model='spherical', coordinate_type='Geographic')
krig.fit(X_krig, y)

X_test_krig[:, 0] = X_test_krig[:, 0] + 360
krig_predictions = krig.predict(X_test_krig)

In [24]:
np.percentile(krig_predictions, 25), np.percentile(krig_predictions, 50), np.percentile(krig_predictions, 75), np.mean(krig_predictions)

(42.34144802768469, 52.5939196233051, 60.12069512413621, 51.19120519687345)